# Discursos da Presidência – o que os torna únicos?
Esse é o código que gera os arquivos .csv por trás da  matéria que analisa as palavras mais características de todos os discursos de posse presidencial no Brasil, desde a proclamação da República até Jair Bolsonaro.

#### Importar os pacotes necessários

In [1]:
# Análise de dados
import pandas as pd

# Gestão de arquivos
import glob

# Tokenização e análise textual
import nltk
from nltk import tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

#### Ler os arquivos com os discursos
Cada discurso de posse foi salvo em um arquivo .txt cujo título é o nome do presidente. Vamos lê-los, um a a um, e salvar em um único dataframe.

In [2]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [3]:
find_fpaths('../output/txts-limpos/', '*.txt')[0][-8:-4]

'1906'

In [4]:
def make_df(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [open(file).read() for file in file_list]
    
    # Transforma eum dataframe
    df = pd.DataFrame({
        'ARQUIVO': file_list,
        'CONTEUDO': content,
        'PRESIDENTE': [ file[22 : -9] for file in file_list ],
        'ANO': [ file[-8 : -4] for file in file_list ]
    })

    return df

#### Tokenizar e vetorizar
Agora, vamos usar os pacotes TextBlob e NLTK para separar os blocos de texto em palavras e contar a ocorrência de cada uma.

In [5]:
def textblob_tokenizer(str_input):
    '''
    >> DESCRIÇÃO

    Tokenizador do TextBlob que separa a string
    que recebe em palvavras individuais. Retorna
    uma lista de palavras.
    
    >> PARÂMETROS
    
    str_input -> O bloco de texto que deve ser tokenizado
    
    min_len -> Opcional. Determina o tamanho mínimo das
    palavras tokenizadas. Todas as palavras cujo comprimento
    seja inferior a este valor são ignoradas.   
    '''
   
    blob = TextBlob( str_input.lower() )
    
    tokens = blob.words
   
    # Pega apenas palavras maiores que 2 letras e que não sejam numéricas
    # O valor está "hard-coded" porque a função não é chamada,
    # mas passada como parâmetro para o vetorizador
    words = [ token for token in tokens if ( len(token) > 2 ) and ( not token.isnumeric() )] 
    
    return words

In [6]:
def make_matrix(df):
    '''
    >> DESCRIÇÃO
    
    Cria uma matriz numérica usando as entradas de texto do
    dataframe e o tokenizador definido anteriormente.
    
    >> PARÂMETROS
    
    df -> O dataframe que contém o discurso dos presidenciáveis.    
    '''
    
    # Define as 'stopwords' usando a lista pronta do NLTK.
    stop_words = nltk.corpus.stopwords.words( 'portuguese' )
    
    # TO DO
    # Adicionar saudações comuns ("excelentíssimo", "exa", "excelência" na lista de stopwords)
    custom_stop_words = [
        'excelentíssimo', 'excelentíssimos', 'excelentíssima', 'excelentíssimas', 'excelência', 'excelências', 'exa', 'exas',
        'senhores', 'senhoras', 'senhor', 'senhora', 'srs', 'sras', 'sr', 'sra'
    ]
    stop_words.extend(custom_stop_words)
    
    # Cria um vetorizador de contagem 
    vec = CountVectorizer( tokenizer = textblob_tokenizer, stop_words = stop_words )
    
    # Fit transform para criar a matriz
    matrix = vec.fit_transform( df.CONTEUDO )
    
    # E cria um dataframe a partir dela, com os valores reais
    matrix = pd.DataFrame( matrix.toarray(), columns=vec.get_feature_names() )

    return matrix

In [7]:
def transpose(df):
    '''
    >> DESCRIÇÃO
    
    Essa função existe apenas para melhor a legibilidade
    do código. Ela usa o método built-in do pandas para
    inverter os eixos do dataframe passado como parâmetro.
    
    >> PARÂMETROS
    
    df -> O dataframe que deve passar pela transposição
    
    '''
    
    # Cópia local para evitar modificações inplace
    df_ = df.copy()
    
    df_ = df_.transpose().reset_index()
    
    return df_

In [8]:
def get_sums(df):
    '''
    >> DESCRIÇÃO
    
    A função conta quantas vezes determinada palavra foi usada
    pelo presidente em questão. Trata-se de um envelope que chama
    o método sum() , também built-in no Pandas.
    
    >> PARÂMETROS
    
    df -> O dataframe com as palavras que devem ser somadas.

    '''
    
    df_ = df.copy()
    
    df_['SOMA_PALAVRA'] = df_.sum( axis = 1 )
    
    # Esse valor será o mesmo ao longo de todo o dataframe
    df_['SOMA_TOTAL'] = df_.SOMA_PALAVRA.sum ( axis = 0 )
    
    # Mantém apenas as colunas de interesse
    df_ = df_[ [ 'index', 'SOMA_PALAVRA', 'SOMA_TOTAL']]
        
    return df_

In [9]:
def get_ratios(df, multiplier):
    '''
    >> DESCRIÇÃO
    
    Com base nas somas obtidas em get_sum(df), calcula
    a razão de uso de cada uma das palavras em relação
    ao total de termos usados em cada discurso.
    
    
    >> PARÂMETROS
    
    df -> O dataframe com os campos de soma já calculados 
    
        
    multiplier -> Um inteiro que vai ser o multiplicador da razão
    calculada – por exemplo, 1 ocorroência a cada 10.000 ou 20.000 palavras.
    '''
    
    df_ = df.copy()
    
    df_['RATIO_PALAVRA'] = ( df_['SOMA_PALAVRA'] / df_['SOMA_TOTAL'] ) * multiplier
    
    return df_

In [10]:
def merge_dfs(this_df, others_df):
    '''
    >> DESCRIÇÃO
    
    Reúne dois dataframes (o primeiro, refente a um único presiednte,
    o segundo referente a todos os demais) em apenas uma variável.
    
    
    >> PARÂMETROS
    
    this_president -> É o dataframe na esquerda do merge, que contém
    os dados calculados para o presidente em questão.
    
    all_others -> É o dataframe na direita do merge, que contém os
    dados caclulados para todos os demais presidentes.    
    '''
    
    suffixes = [ "_ESTE", "_OUTROS" ]
    
    this_df_   = this_df.copy()
    others_df_ = others_df.copy()
    
    
    # Ao fazer o merge na coluna index, estamos reunindo lado a lado
    # os valores referentes a cada uma das palavras.
    df = this_df.merge(others_df, on = 'index', how = 'outer', suffixes = suffixes)
    
    return df

In [11]:
def get_probabilities(df):
    '''
    >> DESCRIÇÃO
    
    A partir do dataframe resultante da função merge_dfs(a, b),
    calcula a diferença da probabilidade de uma palavra ocorrer
    no discurso de um presidente ou de todos os demais.
    
    
    >> PARÂMETROS
    
    df -> O dataframe resultante da função merge_dfs(a, b) para
    os quais os valores de probabilidade serão calculados.    
    '''
    
    df_ = df.copy()
    
    df_['VEZES_MAIS_PROVAVEL_ESTE']  = df_["RATIO_PALAVRA_ESTE"] / df_["RATIO_PALAVRA_OUTROS"]
    df_['VEZES_MAIS_PROVAVEL_OUTROS'] = df_["RATIO_PALAVRA_OUTROS"] / df_["RATIO_PALAVRA_ESTE"]
    
    return df_

In [12]:
def calculate(df, presidente):
    
    # Separa o df
    this_df   = df[ df.PRESIDENTE == presidente ]
    others_df = df[ df.PRESIDENTE != presidente ]
    
    # Cria matrizes
    this_df   = make_matrix( this_df )
    others_df = make_matrix( others_df )
        
    # Faz transposição
    this_df   = transpose( this_df )
    others_df = transpose( others_df )
            
    # Calcula os campos de soma
    this_df   = get_sums( this_df )
    others_df = get_sums( others_df )
    
    # Calcula os campos de razão
    this_df   = get_ratios( this_df, 10000 )
    others_df = get_ratios( others_df, 10000 )
        
    # Executa o merge
    results = merge_dfs( this_df, others_df )
    
    # Calcula as probabilidades
    results = get_probabilities(results)
    
    # Muda o cabeçalho de 'index' para 'PALAVRA'
    results = results.rename( columns = { 'index':'PALAVRA' } )

    # Cria uma coluna com identificador
    results['PRESIDENTE'] = presidente
    
    # Arredonda
    results = results.round(decimals=5)

    # Derruba palavras não usadas pelo candidato ou usadas exclusivamente por ele, que terão um NaN no meio.
    results = results.dropna()
    
    return results

#### Função que encapsula todas as demais e roda o script

In [20]:
def run_script():
    '''
    >> DESCRIÇÃO
    
    Essa função apenas encapsula todas as outras
    rotinas definidas anteriormente e executa
    o código em um loop, que sempre salva o
    output para um csv.
    '''
    
    # Cria um dataframe com todos os discursis
    df = make_df( find_fpaths( dir_path = '../output/txts-limpos/', pattern = "*.txt" ) )
    
    # Define o caminho para salvar o output
    output_path = "../output/matrizes/"

    # Roda o script dentro de um loop que calcula a matriz e salva
    max_value = 0
    for index, row in df.iterrows():
        print( "Calculando matriz do presidente:", row.PRESIDENTE )
        print( "Essa é a matriz", index + 1 , "de", df.shape[0] )
        
        output = calculate(df, row.PRESIDENTE).reset_index( drop = True )
        print( "Esse é o valor máximo da escala X:", output.VEZES_MAIS_PROVAVEL_ESTE.max() )
        print()


    
        filename = row.ANO + '-' + row.PRESIDENTE + '.csv'
        output.to_csv(output_path + filename, index = False)
        
        if output.VEZES_MAIS_PROVAVEL_ESTE.max() > max_value:
            max_value = output.VEZES_MAIS_PROVAVEL_ESTE.max()
            max_presidente = output.loc[ 0, 'PRESIDENTE' ]

        
    print("O valor máximo de probabilidade é", max_value, "em", max_presidente)
    print("Done!")
    return
        
        

In [21]:
run_script()

Calculando matriz do presidente: afonso-penna
Essa é a matriz 1 de 35
Esse é o valor máximo da escala X: 60.30692

Calculando matriz do presidente: artur-bernardes
Essa é a matriz 2 de 35
Esse é o valor máximo da escala X: 315.56981

Calculando matriz do presidente: campos-sales
Essa é a matriz 3 de 35
Esse é o valor máximo da escala X: 40.61063

Calculando matriz do presidente: castelo-branco
Essa é a matriz 4 de 35
Esse é o valor máximo da escala X: 158.60305

Calculando matriz do presidente: costa-e-silva
Essa é a matriz 5 de 35
Esse é o valor máximo da escala X: 187.11145

Calculando matriz do presidente: deodoro-da-fonseca
Essa é a matriz 6 de 35
Esse é o valor máximo da escala X: 900.90323

Calculando matriz do presidente: dilma-rousseff-1
Essa é a matriz 7 de 35
Esse é o valor máximo da escala X: 76.88035

Calculando matriz do presidente: dilma-rousseff-2
Essa é a matriz 8 de 35
Esse é o valor máximo da escala X: 97.68296

Calculando matriz do presidente: emilio-garrastazu-medic

In [22]:
lula_1 = pd.read_csv("../output/matrizes/")

In [24]:
lula_1[ lula_1.PALAVRA == 'fome' ]

,PALAVRA,SOMA_PALAVRA_ESTE,SOMA_TOTAL_ESTE,RATIO_PALAVRA_ESTE,SOMA_PALAVRA_OUTROS,SOMA_TOTAL_OUTROS,RATIO_PALAVRA_OUTROS,VEZES_MAIS_PROVAVEL_ESTE,VEZES_MAIS_PROVAVEL_OUTROS,PRESIDENTE
445,fome,13.0,2095.0,62.05251,13.0,39983.0,3.25138,19.08496,0.0524,lula-1
